### Dependencies

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

### Data Load

In [2]:
import os
import pandas as pd

# Define the directory path where your data is stored
data_dir = 'op_spam_v1.4/negative_polarity'

# Initialize empty lists to store data
reviews = []
labels = []
folds = []

# Loop through each fold
for fold in range(1, 6):
    
    # Read negative truthful reviews
    truthful_dir = os.path.join(data_dir, 'deceptive_from_MTurk', f'fold{fold}')
    for file in os.listdir(truthful_dir):
        with open(os.path.join(truthful_dir, file), 'r', encoding='utf-8') as f:
            review = f.read()
            reviews.append(review)
            labels.append('deceptive')
            folds.append(fold)
    
    # Read negative deceptive reviews
    deceptive_dir = os.path.join(data_dir, 'truthful_from_Web', f'fold{fold}')
    for file in os.listdir(deceptive_dir):
        with open(os.path.join(deceptive_dir, file), 'r', encoding='utf-8') as f:
            review = f.read()
            reviews.append(review)
            labels.append('truthful')
            folds.append(fold)

# Create a DataFrame
data = pd.DataFrame({'Review': reviews, 'Label': labels, 'Fold': folds})

# Display the first few rows of the DataFrame
print(data.head())


                                              Review      Label  Fold
0  James Chicago; the luxurious nice hotel as it ...  deceptive     1
1  We booked a room at the Hilton Chicago for two...  deceptive     1
2  I wish to express my dissatisfaction with my s...  deceptive     1
3  Hotel is located 1/2 mile from the train stati...  deceptive     1
4  I had high hopes for the Hilton Chicago, but I...  deceptive     1


### Preprocessing

In [3]:
from sklearn.preprocessing import LabelEncoder

# Separate data for folds 1 to 4 (training) and fold 5 (testing)
training_data = data[data['Fold'] != 5]
testing_data = data[data['Fold'] == 5]

# Initialize the label encoder
label_encoder = LabelEncoder()

# Fit and transform the labels for training data
y_train = label_encoder.fit_transform(training_data['Label'])

# Transform the labels for testing data
y_test = label_encoder.transform(testing_data['Label'])

# Training data for folds 1 to 4
X_train = training_data['Review']

# Testing data for fold 5
X_test = testing_data['Review']

print("Label Encoder Classes:")
for original_label, encoded_label in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)):
    print(f"{original_label} -> {encoded_label}")

Label Encoder Classes:
deceptive -> 0
truthful -> 1


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust the number of features as needed

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform the testing data
X_test_tfidf = tfidf_vectorizer.transform(X_test)


### 1. Multinomial naive Bayes 

In [5]:

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Create and train the Multinomial Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)

# Make predictions on the test data
y_pred = nb_classifier.predict(X_test_tfidf)

# Calculate and print the accuracy and classification report
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("\nClassification Report:\n", report)

pred_nB = y_pred

Accuracy: 0.78125

Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.96      0.81        80
           1       0.94      0.60      0.73        80

    accuracy                           0.78       160
   macro avg       0.82      0.78      0.77       160
weighted avg       0.82      0.78      0.77       160



### 2. Logistic regression with Lasso penalty

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline

# Create a Logistic Regression model
logistic_regression = LogisticRegression(penalty='l1', solver='liblinear', random_state=42)

# Define the hyperparameter grid for grid search
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100]  # Adjust these values as needed
}

# Create a GridSearchCV instance
grid_search = GridSearchCV(logistic_regression, param_grid, cv=5)  # You can adjust the number of cross-validation folds

# Fit the grid search to the training data
grid_search.fit(X_train_tfidf, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Fit the final model with the best hyperparameters on all training data
best_logistic_regression = LogisticRegression(penalty='l1', solver='liblinear', random_state=42, C=best_params['C'])
best_logistic_regression.fit(X_train_tfidf, y_train)

# Make predictions on the test data
y_pred = best_logistic_regression.predict(X_test_tfidf)

# Calculate and print the accuracy and classification report
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Best Hyperparameters:", best_params)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", report)

pred_lg = y_pred

Best Hyperparameters: {'C': 10}
Accuracy: 0.84375

Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.81      0.84        80
           1       0.82      0.88      0.85        80

    accuracy                           0.84       160
   macro avg       0.85      0.84      0.84       160
weighted avg       0.85      0.84      0.84       160



### 3. Classification trees

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# Create a Decision Tree model
decision_tree = DecisionTreeClassifier(random_state=42)

# Define hyperparameter grid for grid search
param_grid = {
    'ccp_alpha': [0.001, 0.01, 0.1, 1]  # Adjust these values as needed
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(decision_tree, param_grid, cv=10)  # You can change the number of folds (cv) as needed

# Fit the model to the training data
grid_search.fit(X_train_tfidf, y_train)

# Best hyperparameters
best_params = grid_search.best_params_

# Fit the final model with the best hyperparameters on all training data
best_decision_tree = grid_search.best_estimator_
best_decision_tree.fit(X_train_tfidf, y_train)

# Make predictions on the test data
y_pred = best_decision_tree.predict(X_test_tfidf)

# Evaluate the model and print results
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Best Hyperparameters:", best_params)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", report)

pred_ct = y_pred

Best Hyperparameters: {'ccp_alpha': 0.01}
Accuracy: 0.675

Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.74      0.69        80
           1       0.70      0.61      0.65        80

    accuracy                           0.68       160
   macro avg       0.68      0.68      0.67       160
weighted avg       0.68      0.68      0.67       160



### 4. Random forests

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Create a Random Forest model
random_forest = RandomForestClassifier(random_state=42, oob_score=True)

# Define hyperparameter grid for grid search
param_grid = {
    'n_estimators': [100, 200, 300],  # Adjust these values as needed
    'max_features': ['sqrt', 'log2']  # Adjust these values as needed
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(random_forest, param_grid, cv=10)  # You can change the number of folds (cv) as needed

# Fit the model to the training data
grid_search.fit(X_train_tfidf, y_train)

# Best hyperparameters
best_params = grid_search.best_params_

# Fit the final model with the best hyperparameters on all training data
best_random_forest = grid_search.best_estimator_
best_random_forest.fit(X_train_tfidf, y_train)

# Make predictions on the test data
y_pred = best_random_forest.predict(X_test_tfidf)

# Evaluate the model and print results
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Best Hyperparameters:", best_params)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", report)
print("Out-of-Bag Score:", best_random_forest.oob_score_)

pred_rf = y_pred

Best Hyperparameters: {'max_features': 'log2', 'n_estimators': 300}
Accuracy: 0.8375

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.84      0.84        80
           1       0.84      0.84      0.84        80

    accuracy                           0.84       160
   macro avg       0.84      0.84      0.84       160
weighted avg       0.84      0.84      0.84       160

Out-of-Bag Score: 0.825


# Analyse vragen

###  How does the performance of the generative linear model (multinomial naive Bayes) compare to the discriminative linear model 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

# Train a Multinomial Naive Bayes model
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)

# Calculate ROC curve and AUC for Logistic Regression
logistic_regression_probs = best_logistic_regression.predict_proba(X_test_tfidf)[:, 1]
fpr_lr, tpr_lr, thresholds_lr = roc_curve(y_test, logistic_regression_probs)
auc_lr = auc(fpr_lr, tpr_lr)

# Calculate ROC curve and AUC for Naive Bayes
nb_probs = nb_classifier.predict_proba(X_test_tfidf)[:, 1]
fpr_nb, tpr_nb, thresholds_nb = roc_curve(y_test, nb_probs)
auc_nb = auc(fpr_nb, tpr_nb)

# Plot the ROC curves
plt.figure(figsize=(8, 6))
plt.plot(fpr_lr, tpr_lr, label=f'Logistic Regression (AUC = {auc_lr:.2f})')
plt.plot(fpr_nb, tpr_nb, label=f'Multinomial Naive Bayes (AUC = {auc_nb:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()


###  Is the Random Forest able to improve on the performance of the linear classifiers?

In [14]:
# Calculate accuracy for Logistic Regression
y_pred_lr = best_logistic_regression.predict(X_test_tfidf)
accuracy_lr = accuracy_score(y_test, y_pred_lr)

# Calculate accuracy for Multinomial Naive Bayes
y_pred_nb = nb_classifier.predict(X_test_tfidf)
accuracy_nb = accuracy_score(y_test, y_pred_nb)

# Calculate accuracy for Random Forest
y_pred_rf = best_random_forest.predict(X_test_tfidf)
accuracy_rf = accuracy_score(y_test, y_pred_rf)

print(f'Logistic Regression Accuracy: {accuracy_lr:.2f}')
print(f'Multinomial Naive Bayes Accuracy: {accuracy_nb:.2f}')
print(f'Random Forest Accuracy: {accuracy_rf:.2f}')


Logistic Regression Accuracy: 0.84
Multinomial Naive Bayes Accuracy: 0.78
Random Forest Accuracy: 0.84


### Does performance improve by adding bigram features, instead of using just unigrams?

In [15]:
# Create a TF-IDF vectorizer with bigram features
tfidf_vectorizer_bigram = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)  # Adjust max_features as needed

# Fit and transform the training data with bigram features
X_train_tfidf_bigram = tfidf_vectorizer_bigram.fit_transform(X_train)

# Transform the testing data with bigram features
X_test_tfidf_bigram = tfidf_vectorizer_bigram.transform(X_test)


### 1. Multinomial naive Bayes BIGRAM

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Create and train the Multinomial Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf_bigram, y_train)

# Make predictions on the test data
y_pred = nb_classifier.predict(X_test_tfidf_bigram)

# Calculate and print the accuracy and classification report
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("\nClassification Report:\n", report)

pred_nB_bigram = y_pred

Accuracy: 0.8625

Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.89      0.87        80
           1       0.88      0.84      0.86        80

    accuracy                           0.86       160
   macro avg       0.86      0.86      0.86       160
weighted avg       0.86      0.86      0.86       160



### 2. Logistic regression with Lasso penalty BIGRAM

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline

# Create a Logistic Regression model
logistic_regression = LogisticRegression(penalty='l1', solver='liblinear', random_state=42)

# Define the hyperparameter grid for grid search
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100]  # Adjust these values as needed
}

# Create a GridSearchCV instance
grid_search = GridSearchCV(logistic_regression, param_grid, cv=5)  # You can adjust the number of cross-validation folds

# Fit the grid search to the training data
grid_search.fit(X_train_tfidf_bigram, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Fit the final model with the best hyperparameters on all training data
best_logistic_regression = LogisticRegression(penalty='l1', solver='liblinear', random_state=42, C=best_params['C'])
best_logistic_regression.fit(X_train_tfidf_bigram, y_train)

# Make predictions on the test data
y_pred = best_logistic_regression.predict(X_test_tfidf_bigram)

# Calculate and print the accuracy and classification report
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Best Hyperparameters:", best_params)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", report)

pred_lg_bigram = y_pred

Best Hyperparameters: {'C': 100}
Accuracy: 0.75625

Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.71      0.75        80
           1       0.74      0.80      0.77        80

    accuracy                           0.76       160
   macro avg       0.76      0.76      0.76       160
weighted avg       0.76      0.76      0.76       160



### 3. Classification trees BIGRAM

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# Create a Decision Tree model
decision_tree = DecisionTreeClassifier(random_state=42)

# Define hyperparameter grid for grid search
param_grid = {
    'ccp_alpha': [0.001, 0.01, 0.1, 1]  # Adjust these values as needed
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(decision_tree, param_grid, cv=10)  # You can change the number of folds (cv) as needed

# Fit the model to the training data
grid_search.fit(X_train_tfidf, y_train)

# Best hyperparameters
best_params = grid_search.best_params_

# Fit the final model with the best hyperparameters on all training data
best_decision_tree = grid_search.best_estimator_
best_decision_tree.fit(X_train_tfidf_bigram, y_train)

# Make predictions on the test data
y_pred = best_decision_tree.predict(X_test_tfidf_bigram)

# Evaluate the model and print results
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Best Hyperparameters:", best_params)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", report)

pred_ct_bigram = y_pred

Best Hyperparameters: {'ccp_alpha': 0.01}
Accuracy: 0.6125

Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.45      0.54        80
           1       0.58      0.78      0.67        80

    accuracy                           0.61       160
   macro avg       0.63      0.61      0.60       160
weighted avg       0.63      0.61      0.60       160



### 4. Random forests BIGRAM

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Create a Random Forest model
random_forest = RandomForestClassifier(random_state=42, oob_score=True)

# Define hyperparameter grid for grid search
param_grid = {
    'n_estimators': [100, 200, 300],  # Adjust these values as needed
    'max_features': ['sqrt', 'log2']  # Adjust these values as needed
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(random_forest, param_grid, cv=10)  # You can change the number of folds (cv) as needed

# Fit the model to the training data
grid_search.fit(X_train_tfidf_bigram, y_train)

# Best hyperparameters
best_params = grid_search.best_params_

# Fit the final model with the best hyperparameters on all training data
best_random_forest = grid_search.best_estimator_
best_random_forest.fit(X_train_tfidf_bigram, y_train)

# Make predictions on the test data
y_pred = best_random_forest.predict(X_test_tfidf_bigram)

# Evaluate the model and print results
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Best Hyperparameters:", best_params)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", report)
print("Out-of-Bag Score:", best_random_forest.oob_score_)

pred_rf_bigram = y_pred

Best Hyperparameters: {'max_features': 'log2', 'n_estimators': 300}
Accuracy: 0.83125

Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.80      0.83        80
           1       0.81      0.86      0.84        80

    accuracy                           0.83       160
   macro avg       0.83      0.83      0.83       160
weighted avg       0.83      0.83      0.83       160

Out-of-Bag Score: 0.8234375


### What are the five most important terms (features) pointing towards a fake review? 
### What are the five most important terms (features) pointing towards a genuine review?

In [ ]:
# Get the coefficients (weights) from the logistic regression model
coefficients = best_logistic_regression.coef_

# Get the feature names (e.g., words or terms) from the TF-IDF vectorizer
feature_names = tfidf_vectorizer_bigram.get_feature_names_out()

# Create lists to store features associated with each class
class_0_features = []
class_1_features = []

# Iterate through the coefficients and feature names
for coef, feature in zip(coefficients[0], feature_names):
    if coef < 0:  # Negative coefficients are associated with class 0 (genuine reviews)
        class_0_features.append((feature, -coef))  # Use -coef to make it positive
    else:
        class_1_features.append((feature, coef))

# Sort the features by the absolute value of their coefficients
class_0_features.sort(key=lambda x: x[1], reverse=True)
class_1_features.sort(key=lambda x: x[1], reverse=True)

# Print the top features for each class
print("Top features for class 0 (genuine reviews):")
for feature, coef in class_0_features[:5]:  # Change the number to get more or fewer features
    print(f"{feature}: {coef:.4f}")

print("\nTop features for class 1 (fake reviews):")
for feature, coef in class_1_features[:5]:  # Change the number to get more or fewer features
    print(f"{feature}: {coef:.4f}")

In [ ]:
features = tfidf_vectorizer.get_feature_names_out()

feature_coefficients_mult = dict(zip(features, nb_classifier.feature_log_prob_[0]))
feature_coefficients_lg = dict(zip(features, best_logistic_regression.coef_[0]))
feature_coefficients_dt = dict(zip(features, best_decision_tree.feature_importances_))
feature_coefficients_rf = dict(zip(features, best_random_forest.feature_importances_))

top_fake_terms_mult = sorted(feature_coefficients_mult.items(), key=lambda x: x[1], reverse=True)[:5]
top_fake_terms_lg = sorted(feature_coefficients_lg.items(), key=lambda x: x[1], reverse=True)[:5]
top_fake_terms_dt = sorted(feature_coefficients_dt.items(), key=lambda x: x[1], reverse=True)[:5]
top_fake_terms_rf = sorted(feature_coefficients_rf.items(), key=lambda x: x[1], reverse=True)[:5]

top_true_terms_mult = sorted(feature_coefficients_mult.items(), key=lambda x: x[1], reverse=False)[:5]
top_true_terms_lg = sorted(feature_coefficients_lg.items(), key=lambda x: x[1], reverse=False)[:5]
top_true_terms_dt = sorted(feature_coefficients_dt.items(), key=lambda x: x[1], reverse=False)[:5]
top_true_terms_rf = sorted(feature_coefficients_rf.items(), key=lambda x: x[1], reverse=False)[:5]

In [ ]:
print('TOP FAKE TERMS\n')
      
print(f'Multinomial nB:\n{[x[0] for x in top_fake_terms_mult]}\n')
print(f'Logstic regression:\n{[x[0] for x in top_fake_terms_lg]}\n')
print(f'Classification tree:\n{[x[0] for x in top_fake_terms_dt]}\n')
print(f'Random forest:\n{[x[0] for x in top_fake_terms_rf]}\n')

In [ ]:
print('TOP GENUINE TERMS\n')
      
print(f'Multinomial nB:\n{[x[0] for x in top_true_terms_mult]}\n')
print(f'Logstic regression:\n{[x[0] for x in top_true_terms_lg]}\n')
print(f'Classification tree:\n{[x[0] for x in top_true_terms_dt]}\n')
print(f'Random forest:\n{[x[0] for x in top_true_terms_rf]}\n')

## Statistical Tests 

In [21]:
from scipy.stats import chi2_contingency

# Convert to np arrays for efficient computations
pred_nB = np.array(pred_nB)
pred_lg = np.array(pred_lg)
pred_ct = np.array(pred_ct)
pred_rf = np.array(pred_rf)

pred_nB_bigram = np.array(pred_nB_bigram)
pred_lg_bigram = np.array(pred_lg_bigram)
pred_ct_bigram = np.array(pred_ct_bigram)
pred_rf_bigram = np.array(pred_rf_bigram)

y_test = np.array(y_test)

### Generative vs Discriminative (nB vs logistic regression)

In [22]:
from scipy.stats import chi2_contingency

# Calculate four possible options for evaluating two classifiers using McNemar test
a = np.sum((pred_nB == y_test) & (pred_lg == y_test)) # single tree correct, bagged trees correct
b = np.sum((pred_nB == y_test) & (pred_lg != y_test)) # single tree correct, bagged trees incorrect
c = np.sum((pred_nB != y_test) & (pred_lg == y_test)) # single tree incorrect, bagged trees correct
d = np.sum((pred_nB != y_test) & (pred_lg != y_test)) # single tree incorrect, bagged trees incorrect

# Perform statistical test (McNemar)
contingency_table = np.array([[a, b], [c, d]])
chi2, p_value, _, _ = chi2_contingency(contingency_table, correction=True)

# Print results
print("a:", a)
print("b:", b)
print("c:", c)
print("d:", d)
print("McNemar Chi-Square Statistic:", chi2)
print("P-value:", p_value)

# Evaluate statistical significance
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference between the two classifiers.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference between the two classifiers.")

a: 113
b: 12
c: 22
d: 13
McNemar Chi-Square Statistic: 13.714285714285714
P-value: 0.00021282941901437717
Reject the null hypothesis: There is a significant difference between the two classifiers.


### Random Forest vs. linear classifiers

In [23]:
# Calculate four possible options for evaluating two classifiers using McNemar test
a = np.sum((pred_rf == y_test) & (pred_nB == y_test)) # single tree correct, bagged trees correct
b = np.sum((pred_rf == y_test) & (pred_nB != y_test)) # single tree correct, bagged trees incorrect
c = np.sum((pred_rf != y_test) & (pred_nB == y_test)) # single tree incorrect, bagged trees correct
d = np.sum((pred_rf != y_test) & (pred_nB != y_test)) # single tree incorrect, bagged trees incorrect

# Perform statistical test (McNemar)
contingency_table = np.array([[a, b], [c, d]])
chi2, p_value, _, _ = chi2_contingency(contingency_table, correction=True)

# Print results
print("a:", a)
print("b:", b)
print("c:", c)
print("d:", d)
print("McNemar Chi-Square Statistic:", chi2)
print("P-value:", p_value)

# Evaluate statistical significance
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference between the two classifiers.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference between the two classifiers.")

a: 113
b: 21
c: 12
d: 14
McNemar Chi-Square Statistic: 16.40150893882237
P-value: 5.1244419206563755e-05
Reject the null hypothesis: There is a significant difference between the two classifiers.


In [24]:
# Calculate four possible options for evaluating two classifiers using McNemar test
a = np.sum((pred_rf == y_test) & (pred_lg == y_test)) # single tree correct, bagged trees correct
b = np.sum((pred_rf == y_test) & (pred_lg != y_test)) # single tree correct, bagged trees incorrect
c = np.sum((pred_rf != y_test) & (pred_lg == y_test)) # single tree incorrect, bagged trees correct
d = np.sum((pred_rf != y_test) & (pred_lg != y_test)) # single tree incorrect, bagged trees incorrect

# Perform statistical test (McNemar)
contingency_table = np.array([[a, b], [c, d]])
chi2, p_value, _, _ = chi2_contingency(contingency_table, correction=True)

# Print results
print("a:", a)
print("b:", b)
print("c:", c)
print("d:", d)
print("McNemar Chi-Square Statistic:", chi2)
print("P-value:", p_value)

# Evaluate statistical significance
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference between the two classifiers.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference between the two classifiers.")

a: 121
b: 13
c: 14
d: 12
McNemar Chi-Square Statistic: 19.269124463154313
P-value: 1.1352745479056956e-05
Reject the null hypothesis: There is a significant difference between the two classifiers.


### Unigram vs. Bigram

In [26]:
# Calculate four possible options for evaluating two classifiers using McNemar test
a = np.sum((pred_nB == y_test) & (pred_nB_bigram == y_test)) # single tree correct, bagged trees correct
b = np.sum((pred_nB == y_test) & (pred_nB_bigram != y_test)) # single tree correct, bagged trees incorrect
c = np.sum((pred_nB != y_test) & (pred_nB_bigram == y_test)) # single tree incorrect, bagged trees correct
d = np.sum((pred_nB != y_test) & (pred_nB_bigram != y_test)) # single tree incorrect, bagged trees incorrect

# Perform statistical test (McNemar)
contingency_table = np.array([[a, b], [c, d]])
chi2, p_value, _, _ = chi2_contingency(contingency_table, correction=True)

# Print results
print("a:", a)
print("b:", b)
print("c:", c)
print("d:", d)
print("McNemar Chi-Square Statistic:", chi2)
print("P-value:", p_value)

# Evaluate statistical significance
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference between the two classifiers.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference between the two classifiers.")

a: 119
b: 6
c: 19
d: 16
McNemar Chi-Square Statistic: 35.22348955392434
P-value: 2.9395648821751136e-09
Reject the null hypothesis: There is a significant difference between the two classifiers.


In [29]:
# Calculate four possible options for evaluating two classifiers using McNemar test
a = np.sum((pred_lg == y_test) & (pred_lg_bigram == y_test)) # single tree correct, bagged trees correct
b = np.sum((pred_lg == y_test) & (pred_lg_bigram != y_test)) # single tree correct, bagged trees incorrect
c = np.sum((pred_lg != y_test) & (pred_lg_bigram == y_test)) # single tree incorrect, bagged trees correct
d = np.sum((pred_lg != y_test) & (pred_lg_bigram != y_test)) # single tree incorrect, bagged trees incorrect

# Perform statistical test (McNemar)
contingency_table = np.array([[a, b], [c, d]])
chi2, p_value, _, _ = chi2_contingency(contingency_table, correction=True)

# Print results
print("a:", a)
print("b:", b)
print("c:", c)
print("d:", d)
print("McNemar Chi-Square Statistic:", chi2)
print("P-value:", p_value)

# Evaluate statistical significance
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference between the two classifiers.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference between the two classifiers.")

a: 119
b: 16
c: 2
d: 23
McNemar Chi-Square Statistic: 69.22370558734195
P-value: 8.790510277839382e-17
Reject the null hypothesis: There is a significant difference between the two classifiers.


In [27]:
# Calculate four possible options for evaluating two classifiers using McNemar test
a = np.sum((pred_ct == y_test) & (pred_ct_bigram == y_test)) # single tree correct, bagged trees correct
b = np.sum((pred_ct == y_test) & (pred_ct_bigram != y_test)) # single tree correct, bagged trees incorrect
c = np.sum((pred_ct != y_test) & (pred_ct_bigram == y_test)) # single tree incorrect, bagged trees correct
d = np.sum((pred_ct != y_test) & (pred_ct_bigram != y_test)) # single tree incorrect, bagged trees incorrect

# Perform statistical test (McNemar)
contingency_table = np.array([[a, b], [c, d]])
chi2, p_value, _, _ = chi2_contingency(contingency_table, correction=True)

# Print results
print("a:", a)
print("b:", b)
print("c:", c)
print("d:", d)
print("McNemar Chi-Square Statistic:", chi2)
print("P-value:", p_value)

# Evaluate statistical significance
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference between the two classifiers.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference between the two classifiers.")

a: 77
b: 31
c: 21
d: 31
McNemar Chi-Square Statistic: 12.858662075251939
P-value: 0.0003359206779047179
Reject the null hypothesis: There is a significant difference between the two classifiers.


In [28]:
# Calculate four possible options for evaluating two classifiers using McNemar test
a = np.sum((pred_rf == y_test) & (pred_rf_bigram == y_test)) # single tree correct, bagged trees correct
b = np.sum((pred_rf == y_test) & (pred_rf_bigram != y_test)) # single tree correct, bagged trees incorrect
c = np.sum((pred_rf != y_test) & (pred_rf_bigram == y_test)) # single tree incorrect, bagged trees correct
d = np.sum((pred_rf != y_test) & (pred_rf_bigram != y_test)) # single tree incorrect, bagged trees incorrect

# Perform statistical test (McNemar)
contingency_table = np.array([[a, b], [c, d]])
chi2, p_value, _, _ = chi2_contingency(contingency_table, correction=True)

# Print results
print("a:", a)
print("b:", b)
print("c:", c)
print("d:", d)
print("McNemar Chi-Square Statistic:", chi2)
print("P-value:", p_value)

# Evaluate statistical significance
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference between the two classifiers.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference between the two classifiers.")

a: 121
b: 13
c: 12
d: 14
McNemar Chi-Square Statistic: 27.185760037291853
P-value: 1.8481490480307533e-07
Reject the null hypothesis: There is a significant difference between the two classifiers.
